# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [1]:
# Imports here

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# import matplotlib.pyplot as plt

import time
import h5py
import copy

import torch
from torchvision import datasets, transforms, models

from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from collections import OrderedDict

vgg16 = models.vgg16(pretrained=True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:19<00:00, 28745291.24it/s]


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
    'training_transforms': transforms.Compose([
        transforms.RandomRotation(30), 
        transforms.RandomResizedCrop(100), 
        transforms.RandomHorizontalFlip(), 
        transforms.Resize(255), 
        transforms.CenterCrop(224), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'valid_transforms': transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'test_transforms': transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
}




In [3]:
training_transforms = transforms.Compose([transforms.RandomRotation(30), 
                                         transforms.RandomResizedCrop(100), 
                                         transforms.RandomHorizontalFlip(), 
                                         transforms.Resize(255), 
                                         transforms.CenterCrop(224), 
                                         transforms.ToTensor(), 
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255), 
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])

In [4]:
# TODO: Load the datasets with ImageFolder
trainset = datasets.ImageFolder(train_dir, transform=training_transforms)
validset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
testset = datasets.ImageFolder(test_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)


trainset_size = len(trainset)
validset_size = len(validset)
testset_size = len(testset)


class_name = trainset.classes


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [5]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [6]:
# TODO: Build and train your network

# Load a pre-trained network (If you need a starting point, the VGG networks work great and are 
# straightforward to use)
model = vgg16


# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
for param in model.parameters():
    param.requires_grad = False

# from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 512)),
    ('relu', nn.ReLU()),
    # ('drop', nn.Dropout(drop_p)),
    ('fc2', nn.Linear(512, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))
    
model.classifier = classifier


In [7]:


# Train the classifier layers using backpropagation using the pre-trained network to get the features
crit = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
opt = optim.Adam(model.classifier.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 4 epochs
sched = lr_scheduler.StepLR(opt, step_size=4, gamma=0.1)

# Number of epochs
eps = 3  # 10

# Dropout
drop = 0.5

# Phase
ph = 'train'


In [8]:
# TODO: Save the checkpoint 

def save_checkpoint(model, optimizer, epoch, step):
    # model.cpu()
    model.class_to_idx = trainset.class_to_idx
        
    torch.save({
        'arch': 'vgg16',
        'epoch': epoch,
        'step': step,
        'input_size': 25088,
        'output_size': 102,
        'optim_dict': optimizer.state_dict(),
        'state_dict': model.state_dict(),
        'classifier': model.classifier,
        'class_to_idx': model.class_to_idx},
        'curr_model.pth')
    
    
def load_checkpoint():
    model = models.vgg16(pretrained=True)
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    for param in model.parameters():
        param.requires_grad = False
    
    checkpoint = torch.load('curr_model.pth')
    model.class_to_idx = checkpoint['class_to_idx']
    
    model.classifier = checkpoint['classifier']
    optimizer.load_state_dict(checkpoint['optim_dict'])
    model.load_state_dict(checkpoint['state_dict'])
    epoch = checkpoint['epoch']
    steps = checkpoint['step']
    
    return model, epoch, steps

In [9]:
def train_model(retrain, model, criteria, optimizer, scheduler, 
                num_e=25, drop_p=0.5, phase = 'train', steps = 0):
    
 
    print_every = 10
    print('Start ', phase)
    
        
    if phase == 'train':            
        loader = trainloader
        setsize = trainset_size
        model.train()
    if phase == 'valid':
        loader = validloader
        setsize = validset_size
        model.eval()
    if phase == 'test':
        loader = testloader
        setsize = testset_size
        model.eval()
        
    
    for e in range(num_e):
        running_loss = 0
        running_corrects = 0
        
        
        if phase == 'train':
            if retrain == False:
                checkpoint = torch.load('curr_model.pth')
                l_epoch = checkpoint['epoch']
                print('epoch eval: loaded epoch = ', l_epoch)
                if e < l_epoch:
                    print('e: ', e, 'l_epoch: ', l_epoch, '. Setting e to ', l_epoch, '.')
                    e = l_epoch
        
        
        for images, labels in iter(loader):
            steps += 1
            
            
            # WITH THIS SECTION COMMENTED, TRAINING LOSS IMPROVES AS SHOWN BELOW
            # WHEN THIS SECTION IS UN-COMMENTED, THE TRAINING LOSS DOES NOT IMPROVE
            """             
            if phase == 'train':
                if retrain == False:
                    checkpoint = torch.load('curr_model.pth')
                    l_epoch = checkpoint['epoch']
                    l_steps = checkpoint['step']
                    print('steps eval: loaded (current) epoch = ', l_epoch, '. (starting) loaded steps = ', l_steps)
                    if steps != l_steps:
                        print('steps: ', steps, 'l_steps: ', l_steps, '. Setting steps to ', l_steps, '.')
                    steps = l_steps
                
                    model = vgg16
                    optimizer = opt
                    for param in model.parameters():
                        param.requires_grad = False
                    
                    model.class_to_idx = checkpoint['class_to_idx']
                    model.classifier = checkpoint['classifier']
                    optimizer.load_state_dict(checkpoint['optim_dict'])
                    model.load_state_dict(checkpoint['state_dict'])            
            """ 
            
            
            start = time.time()
            
            optimizer.zero_grad()
            
            # Forward and backward passes
            output = model.forward(images)
                
            if phase == 'train':
                                    
                loss = criteria(output, labels)        
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
            
                save_checkpoint(model, optimizer, e, steps + 1)
                if retrain == True:
                    retrain = False
                
                print('model state saved after ' + str(steps) + " steps")
                
            
            if phase == 'valid':
                _, preds = torch.max(output, 1)
                running_corrects += torch.sum(preds == labels.data)
                acc = running_corrects / steps
            
            if steps % print_every == 0:
                
                if phase == 'train':
                
                    print("Steps: ", steps,
                          "Time: ", time.time() - start,
                          "Epoch: {}/{}... ".format(e+1, num_e),
                          "Loss: {:.4f}".format(running_loss/print_every))
                    
                    running_loss = 0
                    
                
                if phase == 'valid':
                    print("Steps: ", steps,
                          "Time: ", time.time() - start,
                          "Epoch: {}/{}... ".format(e+1, num_e),
                          "Acc: {:.4f}".format(acc))
            

In [ ]:
# retrain is variable to either start training from scratch (True) or continue
# training that was in progress (False)
retrain = True

model_ft = train_model(retrain, model, crit, opt, sched, eps, drop, ph, 0)


Start  train
model state saved after 1 steps
model state saved after 2 steps
model state saved after 3 steps
model state saved after 4 steps
model state saved after 5 steps
model state saved after 6 steps
model state saved after 7 steps
model state saved after 8 steps
model state saved after 9 steps
model state saved after 10 steps
Steps:  10 Time:  33.830034255981445 Epoch: 1/3...  Loss: 4.8765
model state saved after 11 steps
model state saved after 12 steps
model state saved after 13 steps
model state saved after 14 steps
model state saved after 15 steps
model state saved after 16 steps
model state saved after 17 steps
model state saved after 18 steps
model state saved after 19 steps
model state saved after 20 steps
Steps:  20 Time:  33.88726878166199 Epoch: 1/3...  Loss: 4.0917
model state saved after 21 steps
model state saved after 22 steps
model state saved after 23 steps
model state saved after 24 steps
model state saved after 25 steps
model state saved after 26 steps
model sta

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [10]:
# TODO: Do validation on the test set

ph = 'valid'
eps = 1
model_ft = train_model(model, crit, opt, sched, eps, drop, ph)

Steps:  10 Time:  18.387450456619263 Epoch: 1/1...  Acc: 21.0000
Steps:  20 Time:  18.42763876914978 Epoch: 1/1...  Acc: 20.0000
Steps:  30 Time:  18.385088682174683 Epoch: 1/1...  Acc: 20.0000
Steps:  40 Time:  18.397141695022583 Epoch: 1/1...  Acc: 20.0000
Steps:  50 Time:  18.354941606521606 Epoch: 1/1...  Acc: 20.0000


KeyboardInterrupt: 

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

torch.save(model, '/home/trained_model.pth')

fc_model = torch.load('/home/trained_model.pth')

In [16]:
save_result = save_checkpoint(model, eps)


In [21]:
load_model = load_checkpoint(mod, eps, stps)

NameError: name 'checkpoint' is not defined

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [1]:
# TODO: Write a function that loads a checkpoint and rebuilds the model



# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes